## Advanced RAG - Data Extraction Pipeline
### Extract PDFs to Markdown, Images, and Tables

**Learning Objectives:**
- Extract PDF content to markdown format
- Save all figures as PNG images
- Extract tables with context (2 paragraphs before)
- Organize extracted content systematically

**Output Structure:**
- Markdown files: Full document text
- Images: `data/rag-images/{company}/{filename}/image_1.png`
- Tables: `data/rag-tables/{company}/{filename}/table_1.md`

### Setup and Imports

In [1]:
from pathlib import Path
from typing import List, Tuple

from docling_core.types.doc import PictureItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

### Configuration

In [2]:
# Paths
DATA_DIR = "data/rag-data"
OUTPUT_MD_DIR = "data/rag-markdown"
OUTPUT_IMAGES_DIR = "data/rag-images"
OUTPUT_TABLES_DIR = "data/rag-tables"

### Extract Metadata from Filename

In [3]:
def extract_metadata_from_filename(filename: str) -> dict:
    """
    Extract metadata from filename.
    
    Expected format: {company} {doc_type} {quarter} {year}.pdf
    Examples:
    - amazon 10-k 2024.pdf
    - amazon 10-q q1 2024.pdf
    """
    name = filename.replace('.pdf', '')
    parts = name.split()
    
    metadata = {}
    metadata['company_name'] = parts[0]
    metadata['doc_type'] = parts[1]
    
    if len(parts) == 4:
        metadata['fiscal_quarter'] = parts[2]
        metadata['fiscal_year'] = int(parts[3])
    else:
        metadata['fiscal_quarter'] = None
        metadata['fiscal_year'] = int(parts[2])
    
    return metadata

### Extract Tables with Context

In [4]:
def extract_tables_with_context(markdown_text: str) -> List[Tuple[str, str]]:
    """
    Extract tables with 2 paragraphs of context before each table.
    
    Returns:
        List of (context + table, table_number) tuples
    """
    # Split by table pattern (markdown tables start with |)
    lines = markdown_text.split('\n')
    
    tables = []
    i = 0
    table_num = 1
    
    while i < len(lines):
        line = lines[i]
        
        # Detect table start (line with multiple |)
        if line.strip().startswith('|') and line.count('|') >= 2:
            # Find 2 paragraphs before
            context_lines = []
            para_count = 0
            j = i - 1
            
            while j >= 0 and para_count < 2:
                if lines[j].strip():  # Non-empty line
                    context_lines.insert(0, lines[j])
                elif context_lines:  # Empty line marks paragraph break
                    para_count += 1
                j -= 1
            
            # Extract full table
            table_lines = []
            while i < len(lines) and (lines[i].strip().startswith('|') or not lines[i].strip()):
                if lines[i].strip():  # Skip empty lines within table
                    table_lines.append(lines[i])
                i += 1
                if i < len(lines) and lines[i].strip() and not lines[i].strip().startswith('|'):
                    break
            
            # Combine context + table
            full_content = '\n'.join(context_lines) + '\n\n' + '\n'.join(table_lines)
            tables.append((full_content, f"table_{table_num}"))
            table_num += 1
        else:
            i += 1
    
    return tables

### Extract PDF Content

In [ ]:
def extract_pdf_content(pdf_path: Path):
    """Extract PDF to markdown, images, and tables."""
    print(f"Processing: {pdf_path.name}")
    
    # Get metadata and create directories
    metadata = extract_metadata_from_filename(pdf_path.name)
    company = metadata['company_name']
    filename_stem = pdf_path.stem
    
    md_dir = Path(OUTPUT_MD_DIR) / company
    images_dir = Path(OUTPUT_IMAGES_DIR) / company / filename_stem
    tables_dir = Path(OUTPUT_TABLES_DIR) / company / filename_stem
    
    for dir_path in [md_dir, images_dir, tables_dir]:
        dir_path.mkdir(parents=True, exist_ok=True)
    
    # Configure and convert
    pipeline_options = PdfPipelineOptions()
    pipeline_options.generate_picture_images = True
    pipeline_options.generate_page_images = True
    
    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
    )
    result = converter.convert(str(pdf_path))
    
    # Save markdown
    page_break = "<!-- page break -->"
    markdown_text = result.document.export_to_markdown(page_break_placeholder=page_break)
    (md_dir / f"{filename_stem}.md").write_text(markdown_text, encoding='utf-8')
    print(f"  ✓ Markdown saved")
    
    # Find pages with large images and save them
    pages_to_save = set()
    for element, _ in result.document.iterate_items():
        if isinstance(element, PictureItem):
            image = element.get_image(result.document)
            if image.size[0] > 500 and image.size[1] > 500:
                page_no = element.prov[0].page_no if element.prov else None
                if page_no:
                    pages_to_save.add(page_no)
    
    # Save page images
    for page_no in pages_to_save:
        page = result.document.pages[page_no]
        page.image.pil_image.save(images_dir / f"page_{page_no}.png", "PNG")
    
    if pages_to_save:
        print(f"  ✓ Saved {len(pages_to_save)} page images")
    
    # Save tables with context
    tables = extract_tables_with_context(markdown_text)
    for table_content, table_name in tables:
        (tables_dir / f"{table_name}.md").write_text(table_content, encoding='utf-8')
    
    if tables:
        print(f"  ✓ Saved {len(tables)} tables")
    
    print(f"  [DONE]\n")

### Process All PDFs

In [6]:
# Find all PDF files
data_path = Path(DATA_DIR)
pdf_files = list(data_path.rglob("*.pdf"))
print(f"Found {len(pdf_files)} PDF files\n")

# Process each PDF
for pdf_path in pdf_files:
    extract_pdf_content(pdf_path)

2025-12-12 19:53:42,171 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-12 19:53:42,192 - INFO - Going to convert document batch...
2025-12-12 19:53:42,192 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f3d5bd58852fb93aafa57fe606de9c74
2025-12-12 19:53:42,212 - INFO - Loading plugin 'docling_defaults'
2025-12-12 19:53:42,214 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-12 19:53:42,233 - INFO - Loading plugin 'docling_defaults'
2025-12-12 19:53:42,237 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']


Found 28 PDF files

Processing: amazon 10-k 2023.pdf


2025-12-12 19:53:42,400 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-12 19:53:42,409 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-12 19:53:42,413 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-12 19:53:42,414 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-12 19:53:42,463 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-12 19:53:42,464 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-12-12 19:53:42,465 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-12-12 19:53:42,489 [RapidOCR] base.py:22: Using engine_name: onn

  ✓ Saved markdown: data\rag-markdown\amazon\amazon 10-k 2023.md
  ✓ Saved 61 tables (text) to: data\rag-tables\amazon\amazon 10-k 2023
  [DONE] amazon 10-k 2023.pdf

Processing: amazon 10-k 2024.pdf


2025-12-12 19:54:13,859 - INFO - Auto OCR model selected rapidocr with onnxruntime.
2025-12-12 19:54:13,860 - INFO - Accelerator device: 'cuda:0'
2025-12-12 19:54:14,962 - INFO - Accelerator device: 'cuda:0'
2025-12-12 19:54:15,270 - INFO - Processing document amazon 10-k 2024.pdf
2025-12-12 19:54:44,746 - INFO - Finished converting document amazon 10-k 2024.pdf in 31.06 sec.
2025-12-12 19:54:44,909 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-12 19:54:44,911 - INFO - Going to convert document batch...
2025-12-12 19:54:44,911 - INFO - Initializing pipeline for StandardPdfPipeline with options hash f3d5bd58852fb93aafa57fe606de9c74
2025-12-12 19:54:44,912 - INFO - Accelerator device: 'cuda:0'
[INFO] 2025-12-12 19:54:44,921 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-12 19:54:44,924 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2025-12-

...

[INFO] 2025-12-12 20:02:27,404 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-12-12 20:02:27,404 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2025-12-12 20:02:27,429 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-12 20:02:27,436 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_rec_infer.onnx
[INFO] 2025-12-12 20:02:27,436 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_rec_infer.onnx
2025-12-12 20:02:27,509 - INFO - Auto OCR model selected rapidocr with onnxruntime.
2025-12-12 20:02:27,510 - INFO - Accelerator device: 'cuda:0'
2025-12-12 20:02:28,543 - INFO - Accelerator device: 'cuda:0'
2025-12-12 20:0

  ✓ Saved markdown: data\rag-markdown\meta\meta 10-q q3 2025.md
  ✓ Saved 8 images to: data\rag-images\meta\meta 10-q q3 2025
  ⊘ Skipped 25 small images (< 500x500)
  ✓ Saved 3 tables (text) to: data\rag-tables\meta\meta 10-q q3 2025
  [DONE] meta 10-q q3 2025.pdf

Processing: meta 10-q q4 2024.pdf


[INFO] 2025-12-12 20:02:34,362 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-12 20:02:34,368 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_rec_infer.onnx
[INFO] 2025-12-12 20:02:34,368 [RapidOCR] main.py:53: Using C:\Users\laxmi\anaconda3\envs\ml\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_rec_infer.onnx
2025-12-12 20:02:34,434 - INFO - Auto OCR model selected rapidocr with onnxruntime.
2025-12-12 20:02:34,434 - INFO - Accelerator device: 'cuda:0'
2025-12-12 20:02:35,443 - INFO - Accelerator device: 'cuda:0'
2025-12-12 20:02:35,710 - INFO - Processing document meta 10-q q4 2024.pdf
2025-12-12 20:02:41,232 - INFO - Finished converting document meta 10-q q4 2024.pdf in 6.96 sec.


  ✓ Saved markdown: data\rag-markdown\meta\meta 10-q q4 2024.md
  ✓ Saved 9 images to: data\rag-images\meta\meta 10-q q4 2024
  ⊘ Skipped 20 small images (< 500x500)
  ✓ Saved 3 tables (text) to: data\rag-tables\meta\meta 10-q q4 2024
  [DONE] meta 10-q q4 2024.pdf

Processing: meta10-k 2023.pdf


IndexError: list index out of range

### Verify Extraction

In [ ]:
# Count extracted files
md_files = list(Path(OUTPUT_MD_DIR).rglob("*.md"))
image_files = list(Path(OUTPUT_IMAGES_DIR).rglob("*.png"))
table_files = list(Path(OUTPUT_TABLES_DIR).rglob("*.md"))

print(f"Extraction Summary:")
print(f"  Markdown files: {len(md_files)}")
print(f"  Images: {len(image_files)}")
print(f"  Tables: {len(table_files)}")

### Example: View Extracted Content

In [ ]:
# Show sample markdown
if md_files:
    sample_md = md_files[0]
    print(f"Sample Markdown: {sample_md}\n")
    with open(sample_md, 'r', encoding='utf-8') as f:
        print(f.read()[:500])

# Show sample table
if table_files:
    sample_table = table_files[0]
    print(f"\n\nSample Table: {sample_table}\n")
    with open(sample_table, 'r', encoding='utf-8') as f:
        print(f.read())